# Catan Draft Simulator

by Ryan Fernandes

### Imports

In [ ]:
import math
import time
import random
import heapq
import numpy as np
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import Optional, Callable, List
from enum import IntEnum
from __future__ import annotations
import pandas as pd
from IPython.display import clear_output

### Constants

In [ ]:
class Resource(IntEnum):
    DESERT = 0
    WOOD = 1
    BRICK = 2
    WHEAT = 3
    SHEEP = 4
    ORE = 5

class TileCounts(IntEnum):
    DESERT = 1
    WOOD = 4
    BRICK = 3
    WHEAT = 4
    SHEEP = 4
    ORE = 3

RESOURCE_NAMES = ["DESERT", "WOOD", "BRICK", "WHEAT", "SHEEP", "ORE"]

TILES = [Resource.DESERT for i in range(TileCounts.DESERT)] + [Resource.WOOD for i in range(TileCounts.WOOD)] + [Resource.BRICK for i in range(TileCounts.BRICK)] + [Resource.WHEAT for i in range(TileCounts.WHEAT)] + [Resource.SHEEP for i in range(TileCounts.SHEEP)] + [Resource.ORE for i in range(TileCounts.ORE)]
NUMBERS = list(range(3, 7)) * 2 + list(range(8, 12)) * 2 + [2, 12]
ORDERED_NUMBERS = [5, 2, 6, 3, 8, 10, 9, 12, 11, 4, 8, 10, 9, 4, 5, 6, 3, 11]

HEX_VERTICES = [
    [0, 1, 28, 29, 30, 47],
    [1, 2, 3, 30, 31, 32],
    [3, 4, 5, 6, 32, 33],
    [6, 7, 8, 33, 34, 35],
    [8, 9, 10, 11, 35, 36],
    [11, 12, 13, 36, 37, 38],
    [13, 14, 15, 16, 38, 39],
    [16, 17, 18, 39, 40, 41],
    [18, 19, 20, 21, 41, 42],
    [21, 22, 23, 42, 43, 44],
    [23, 24, 25, 26, 44, 45],
    [26, 27, 28, 45, 46, 47],
    [30, 31, 46, 47, 48, 53],
    [31, 32, 33, 34, 48, 49],
    [34, 35, 36, 37, 49, 50],
    [37, 38, 39, 40, 50, 51],
    [40, 41, 42, 43, 51, 52],
    [43, 44, 45, 46, 52, 53],
    [48, 49, 50, 51, 52, 53]
]

VERTEX_HEXES = [[] for i in range(54)]

for index, row in enumerate(HEX_VERTICES):
    for vertex in row:
        VERTEX_HEXES[vertex].append(index)

VERTEX_NEIGHBORS = [
    [1, 29],
    [0, 2, 30],
    [1, 3],
    [2, 4, 32],
    [3, 5],
    [4, 6],
    [5, 7, 33],
    [6, 8],
    [7, 9, 35],
    [8, 10],
    [9, 11],
    [10, 12, 36],
    [11, 13],
    [12, 14, 38],
    [13, 15],
    [14, 16],
    [15, 17, 39],
    [16, 18],
    [17, 19, 41],
    [18, 20],
    [19, 21],
    [20, 22, 42],
    [21, 23],
    [22, 24, 44],
    [23, 25],
    [24, 26],
    [25, 27, 45],
    [26, 28],
    [27, 29, 47],
    [0, 28],
    [1, 31, 47],
    [30, 32, 48],
    [3, 31, 33],
    [6, 32, 34],
    [33, 35, 49],
    [8, 34, 36],
    [11, 35, 37],
    [36, 38, 50],
    [13, 37, 39],
    [16, 38, 40],
    [39, 41, 51],
    [18, 40, 42],
    [21, 41, 43],
    [42, 44, 52],
    [23, 43, 45],
    [26, 44, 46],
    [45, 47, 53],
    [28, 30, 46],
    [31, 49, 53],
    [34, 48, 50],
    [37, 49, 51],
    [40, 50, 52],
    [43, 51, 53],
    [46, 48, 52]
]

VERTEX_ADJACENCY = np.array([[0 for i in range(54)] for i in range(54)])

for index, row in enumerate(VERTEX_NEIGHBORS):
    for location in row:
        VERTEX_ADJACENCY[index][location] = 1

VERTEX_PROSPECTS = np.linalg.matrix_power(VERTEX_ADJACENCY, 2)

for i in range(54):
    VERTEX_PROSPECTS[i][i] = 0

EDGES = []

for i in range(54):
    for j in range(i):
        if VERTEX_ADJACENCY[i][j]:
            EDGES.append((i, j))

### Data Structures

In [ ]:
class Hex():
    """
    Resource mappings:
    - Desert -> 0
    - Wood -> 1
    - Brick -> 2
    - Wheat -> 3
    - Sheep -> 4
    - Ore -> 5
    """
    resource: int
    # Number may only be None for deserts
    number: Optional[int] = None
    
    pips: int = 0
    prob: float = 0
    
    def __init__(self, resource, number = None):
        if not 0 <= resource <= 5:
            raise ValueError("The resource must be an integer from 0 to 5")
        if resource != 0:
            if number is None:
                raise ValueError("For all non-desert tiles, there must be a value for number")
            elif not 2 <= number <= 12 or number == 7:
                raise ValueError("Invalid value for number. Must be 2-6 or 8-12")

        self.resource = resource
        if resource == 0:
            self.number = None
        else:
            self.number = number
        self.pips = 0 if resource == 0 else 6 - abs(7 - number)
        self.prob = self.pips / 36

    def set_number(self, number: int):
        if self.resource != 0:
            if number is None:
                raise ValueError("For all non-desert tiles, there must be a value for number")
            elif not 2 <= number <= 12 or number == 7:
                raise ValueError("Invalid value for number. Must be 2-6 or 8-12")
        self.number = number
        self.pips = 0 if self.resource == 0 else 6 - abs(7 - number)
        self.prob = self.pips / 36

    def set_resource(self, resource: int):
        if not 0 <= resource <= 5:
            raise ValueError("The resource must be an integer from 0 to 5")
        self.resource = resource

class Vertex():
    # Locations numbered 0 - 53, going from the outer ring to the center in counter-clockwise fashion
    # Counting always begins in the top-left corner for each ring
    location: int
    hexes: List[Hex]
    
    edges: List[Edge]
    hex_count: int = 0
    pips: int = 0
    resource_counts: List[int]
    resource_pips: List[int]
    neighbors: List[Vertex]
    prospects: List[Vertex]
    available: bool = True
    occupied: bool = False
    occupant: Optional[int] = None
    # 1 = settlement, 2 = city
    occupant_level: Optional[int] = None

    def __init__(self, location, hexes):
        if not 0 <= location <= 53:
            raise ValueError("Location must be from 0 to 53")
        if not 0 <= len(hexes) <= 3:
            raise ValueError("Invalid hexes array: there must be from 0 to 3 hexes for each vertex")
        self.location = location
        self.hexes = hexes
        self.hex_count = len(hexes)
        self.resource_counts = [0 for i in range(6)]
        self.resource_pips = [0 for i in range(6)]
        for item in hexes:
            self.resource_counts[item.resource] += 1
            if item.resource != Resource.DESERT:
                self.resource_pips[item.resource] += item.pips
            self.pips += item.pips
        self.edges = []
        self.neighbors = []
        self.prospects = []

    def make_unavailable(self):
        self.available = False

    def try_available(self):
        if not any([n.occupied for n in self.neighbors]):
            self.available = True

    def add_edge(self, edge: Edge):
        self.edges.append(edge)
    
    def occupy(self, player: int, force: bool = False):
        if self.occupied and not force:
            raise ValueError("Cannot occupy a space that is already occupied. Specify force=True to override")
        if not self.available and not force:
            raise ValueError("Cannot occupy a space that is not available (has distance 1 neighbor). Specify force=True to override")
        
        self.occupied = True
        self.available = False
        self.occupant = player
        self.occupant_level = 1

        for neighbor in self.neighbors:
            neighbor.make_unavailable()

    def vacate(self):
        self.occupied = False
        self.available = True
        self.occupant = None
        self.occupant_level = None

        for neighbor in self.neighbors:
            neighbor.try_available()

    def add_neighbor(self, neighbor: Vertex):
        self.neighbors.append(neighbor)

    def add_prospect(self, prospect: Vertex):
        self.prospects.append(prospect)

    def hex_update(self):
        self.resource_counts = [0 for i in range(6)]
        self.resource_pips = [0 for i in range(6)]
        self.pips = 0
        for item in self.hexes:
            self.resource_counts[item.resource] += 1
            if item.resource != Resource.DESERT:
                self.resource_pips[item.resource] += item.pips
            self.pips += item.pips

    def best_road(self):
        best_edge = self.edges[0]
        adjacent = best_edge.vertices[0] if best_edge.vertices[1].location == self.location else best_edge.vertices[1]
        best_total = 0
        
        for neighbor in adjacent.neighbors:
            if neighbor.available:
                best_total += neighbor.pips

        for edge in self.edges[1:]:
            adjacent = best_edge.vertices[0] if best_edge.vertices[1].location == self.location else best_edge.vertices[1]
            total = 0
            
            for neighbor in adjacent.neighbors:
                if neighbor.available:
                    total += neighbor.pips

            if total > best_total:
                best_total = total
                best_edge = edge

        return best_edge

class Edge():
    vertices: List[Vertex]
    
    occupied: bool = False
    occupant: Optional[int] = None

    road_id: str = ""
    index: int = 0
    
    def __init__(self, vertices, index):
        if not len(vertices) == 2:
            raise ValueError("Edges must have two vertices")
        self.vertices = vertices
        self.road_id = "-".join([str(v.location) for v in self.vertices])
        self.index = index
    
    def occupy(self, player: int, force: bool = False):
        if self.occupied and not force:
            raise ValueError("Cannot occupy an edge that is already occupied. Specify force=True to override")
        
        self.occupied = True
        self.occupant = player

    def vacate(self):
        self.occupied = False
        self.occupant = None
    

class Player():
    number: int
    
    settlements: List[Vertex]
    roads: List[Edge]
    resource_counts: List[int]

    def __init__(self, number):
        self.number = number
        self.settlements = []
        self.roads = []
        self.resource_counts = [0 for i in range(6)]

    def add_settlement(self, settlement: Vertex):
        settlement.occupy(self.number)
        self.settlements.append(settlement)

    def pop_settlement(self):
        settlement = self.settlements.pop()
        settlement.vacate()
        return settlement.location

    def remove_settlement(self, settlement: Vertex):
        prior = len(self.settlements)
        self.settlements = [v for v in self.settlements if v.location != settlement.location]
        if not prior == len(self.settlements):
            settlement.vacate()

    def add_road(self, road: Edge):
        road.occupy(self.number)
        self.roads.append(road)

    def pop_road(self):
        road = self.roads.pop()
        road.vacate()
        return road.road_id

    def remove_road(self, road: Edge):
        prior = len(self.roads)
        self.roads = [e for e in self.roads if e.road_id != road.road_id]
        if not prior == len(self.roads):
            road.vacate()

    def reset(self):
        while(self.roads):
            self.pop_road()
        while(self.settlements):
            self.pop_settlement()
        self.resource_counts = [0 for i in range(6)]

    def total_pips(self):
        total = 0
        for settlement in self.settlements:
            total += settlement.pips
        return total

    def summarize(self):
        print(f"Player {self.number}:")
        print(f"Settlements: {", ".join([str(v.location) for v in settlements])}")
        print(f"Wood: {self.resource_counts[0]}\nBrick: {self.resource_counts[1]}\nWheat: {self.resource_counts[2]}\nSheep: {self.resources_counts[3]}\nOre: {self.resource_counts[4]}")

class Board():
    vertices: List[Vertex]
    edges: List[Edge]
    hexes: List[Hex]

    def __init__(self):
        self.vertices = []
        self.edges = []
        self.hexes = []
        
        for i in range(len(HEX_VERTICES)):
            new_hex = Hex(resource=Resource.DESERT)
            self.hexes.append(new_hex)

        for index, row in enumerate(VERTEX_HEXES):
            v_hexes = [self.hexes[i] for i in row]
            new_vertex = Vertex(location=index, hexes=v_hexes)
            self.vertices.append(new_vertex)
        
        for index, pair in enumerate(EDGES):
            e_vertices = [self.vertices[i] for i in pair]
            new_edge = Edge(vertices=e_vertices, index=index)
            self.edges.append(new_edge)

        for edge in self.edges:
            for vertex in edge.vertices:
                vertex.add_edge(edge)

    def normal_randomize(self):
        resources = [resource for resource in TILES]
        random.shuffle(resources)
        
        offset = 0
        
        for index, curr_hex in enumerate(self.hexes):
            curr_hex.set_resource(resources[index])
            if resources[index] == Resource.DESERT:
                curr_hex.set_number(None)
                offset += 1
            else:
                curr_hex.set_number(ORDERED_NUMBERS[index - offset])

        for vertex in self.vertices:
            vertex.hex_update()

    def randomize(self):
        resources = [resource for resource in TILES]
        numbers = [number for number in NUMBERS]
        random.shuffle(resources)
        random.shuffle(numbers)
        
        offset = 0
        
        for index, curr_hex in enumerate(self.hexes):
            curr_hex.set_resource(resources[index])
            if resources[index] == Resource.DESERT:
                curr_hex.set_number(None)
                offset += 1
            else:
                curr_hex.set_number(numbers[index - offset])

        for vertex in self.vertices:
            vertex.hex_update()

    def set_board(self, resources, numbers):
        offset = 0
        
        for index, curr_hex in enumerate(self.hexes):
            curr_hex.set_resource(resources[index])
            if resources[index] == Resource.DESERT:
                curr_hex.set_number(None)
                offset += 1
            else:
                curr_hex.set_number(numbers[index - offset])

        for vertex in self.vertices:
            vertex.hex_update()

    def summarize(self):
        print("Board summary by hex:")
        for i, curr_hex in enumerate(self.hexes):
            print(f"Hex {i}: {RESOURCE_NAMES[curr_hex.resource]}, {curr_hex.number}")

    def available_vertices(self):
        return [v for v in self.vertices if v.available]

    def populate_neighbors(self):
        for i, row in enumerate(VERTEX_ADJACENCY):
            for j, num in enumerate(row):
                if num == 1:
                    self.vertices[i].add_neighbor(self.vertices[j])

    def populate_prospects(self):
        for i, row in enumerate(VERTEX_PROSPECTS):
            for j, num in enumerate(row):
                if num == 1:
                    self.vertices[i].add_prospect(self.vertices[j])
                    
    def clear(self):
        for vertex in self.vertices:
            if vertex.occupied:
                vertex.vacate()
        for edge in self.edges:
            if edge.occupied:
                edge.vacate()

class DraftSimulator():
    board: Board
    players: List[Player]
    choices: List[Vertex]
    evaluate: Callable[[List[Player], Board], List[float]]
    consider: Callable[[Vertex, Board, int], bool]
    top_k_eval: Callable[[Vertex], int]
    history: []
    chosen: []
    scores: []
    used_k: int
    chosen_roads: []

    def __init__(self, evaluate, consider, top_k_eval):
        self.board = Board()
        self.players = [Player(number=i) for i in range(4)]
        self.evaluate = evaluate
        self.consider = consider
        self.top_k_eval = top_k_eval
        self.choices = []
        self.history = []
        self.chosen = []
        self.used_k = 0;
        self.chosen_roads = []

    def set_eval(self, evaluate):
        self.evaluate = evaluate

    def set_consider(self, consider):
        self.consider = consider

    def set_top_k_eval(self, top_k_eval):
        self.top_k_eval = top_k_eval

    def reset(self):
        for player in self.players:
            player.reset()
    
    def force_choice(self, player: int, location: int):
        self.players[player].add_settlement(self.board.vertices[location])
        
    def pop_choice(self, player: int):
        self.players[player].pop_settlement()

    def force_road(self, player: int, index: int):
        self.players[player].add_road(self.board.edges[index])
        
    def pop_road(self, player: int):
        self.players[player].pop_road()

    def mixed_max_dfs(self, order: List[int]):
        if not order:
            return (self.evaluate(self.players, self.board), [])

        player = order[0]
        max_yield = [-math.inf for i in range(4)]
        max_vertex = None
        max_choices = []

        for vertex in self.board.available_vertices():
            self.force_choice(player, vertex.location)
            results = self.mixed_max_dfs([num for index, num in enumerate(order) if index > 0])
            if results[0][player] >= max_yield[player]:
                max_yield = results[0]
                max_vertex = vertex
                max_choices = results[1]
            self.pop_choice(player)

        return(max_yield, [max_vertex, *max_choices])

    def ruleout_mixed_max_dfs(self, order: List[int]):
        if not order:
            return (self.evaluate(self.players, self.board), [])

        player = order[0]
        max_yield = [-math.inf for i in range(4)]
        max_vertex = None
        max_choices = []

        for vertex in self.board.available_vertices():
            if self.consider(vertex, self.board, len(order)):
                self.force_choice(player, vertex.location)
                results = self.ruleout_mixed_max_dfs([num for index, num in enumerate(order) if index > 0])
                if results[0][player] >= max_yield[player]:
                    max_yield = results[0]
                    max_vertex = vertex
                    max_choices = results[1]
                self.pop_choice(player)

        return(max_yield, [max_vertex, *max_choices])

    def top_k_mixed_max_dfs(self, order: List[int], k):
        if not order:
            return (self.evaluate(self.players, self.board), [], [])

        player = order[0]
        max_yield = [-math.inf for i in range(4)]
        max_vertex = None
        max_choices = []
        
        heap = []

        log = []
        max_logs = []

        for vertex in self.board.available_vertices():
            heapq.heappush(heap, (self.top_k_eval(vertex), vertex.location, vertex))
            if len(heap) > k:
                heapq.heappop(heap)

        while heap:
            package = heapq.heappop(heap)
            vertex = package[2]
            self.force_choice(player, vertex.location)
            results = self.top_k_mixed_max_dfs([num for index, num in enumerate(order) if index > 0], k)

            log.append({
                "location": vertex.location,
                "prospect": self.top_k_eval(vertex),
                "all_locations": [vertex.location, *[v.location for v in results[1]]],
                "yields": [float(r) for r in results[0]]
            })
            
            if results[0][player] >= max_yield[player]:
                max_yield = results[0]
                max_vertex = vertex
                max_choices = results[1]
                max_logs = results[2]
            self.pop_choice(player)
        
        return(max_yield, [max_vertex, *max_choices], [log, *max_logs])
        
    def run_mixed_max(self, order: List[int]):
        result = self.mixed_max_dfs(order)

        for index, vertex in enumerate(result[1]):
            self.force_choice(order[index], vertex.location)

        return result

    def run_ruleout_mixed_max(self, order: List[int]):
        result = self.ruleout_mixed_max_dfs(order)

        for index, vertex in enumerate(result[1]):
            self.force_choice(order[index], vertex.location)

        return result

    def run_top_k_mixed_max(self, order: List[int], k):
        self.history = []
        self.chosen = []
        self.scores = []
        self.used_k = k
        self.chosen_roads = []
        
        result = self.top_k_mixed_max_dfs(order, k)

        for index, vertex in enumerate(result[1]):
            self.force_choice(order[index], vertex.location)

        roads = []

        for index, vertex in enumerate(result[1]):
            best_edge = vertex.best_road()
            self.force_road(order[index], best_edge.index)
            roads.append(best_edge.index)

        res = (result[0], result[1], roads, [player.total_pips() for player in self.players])
        
        self.history = result[2]
        self.scores = [float(s) for s in result[0]]
        self.chosen = [v.location for v in result[1]]
        self.chosen_roads = [road for road in roads]

        return res

    def run_random(self, order: List[int]):
        choices = []
        
        for num in order:
            vertex = random.choice(self.board.available_vertices())
            self.force_choice(num, vertex.location)
            choices.append(vertex)

        result = [choices, [player.total_pips() for player in self.players]]

        return result

    def get_summary(self):
        return {
            "used_k": self.used_k,
            "chosen": self.chosen,
            "chosen_roads": self.chosen_roads,
            "scores": self.scores,
            "history": self.history,
            "board": [
                {
                    "resource": h.resource.value,
                    "number": 7 if h.number is None else h.number
                } for h in self.board.hexes
            ]
        }

## Helper Functions

In [ ]:
def relative_pips(players: List[Player], board: Board):
    pips = [sum([settlement.pips for settlement in player.settlements]) for player in players]

    res = []

    for i in range(len(players)):
        m = np.mean(pips[:i] + pips[i + 1:])
        res.append(pips[i] - m)

    return res

In [ ]:
def relative_max_pips(players: List[Player], board: Board):
    pips = [sum([settlement.pips for settlement in player.settlements]) for player in players]

    res = []

    for i in range(len(players)):
        m = np.max(pips[:i] + pips[i + 1:])
        res.append(pips[i] - m)

    return res

In [ ]:
def fit_pips(player: Player, board: Board):
    pips = [0, 0, 0, 0, 0, 0]
    
    for settlement in player.settlements:
        for index, pip in enumerate(settlement.resource_pips):
            pips[index] += pip

    metric_1 = min(pips[1:5]) * 5
    metric_2 = min([pips[1], pips[2]]) * 2
    metric_3 = min([pips[5] / 3, pips[3] / 2]) * 5

    return max([metric_1, metric_2, metric_3])

def sum_prospects(player: Player, board: Board):
    total = 0

    for settlement in player.settlements:
        for prospect in settlement.prospects:
            if prospect.available:
                total += prospect.pips

    return total

def relative_custom_heuristic(players: List[Player], board: Board):
    pips = [sum([settlement.pips for settlement in player.settlements]) for player in players]

    pip_fit = [fit_pips(player, board) for player in players]

    prospect_sum = [sum_prospects(player, board) for player in players]

    total = []

    for i in range(len(pips)):
        total.append(pips[i] + 0.5 * pip_fit[i] + 0.25 * prospect_sum[i])

    res = []

    for i in range(len(players)):
        m = np.max(total[:i] + total[i + 1:])
        res.append(total[i] - m)

    return res

In [ ]:
def simple_ruleout(vertex: Vertex, board: Board, remaining_turns: int):
    if vertex.hex_count == 1:
        return False
    if remaining_turns >= 6 and vertex.hex_count < 3:
        return False
    if vertex.pips <= 7:
        return False

    return True

In [ ]:
def num_pips(vertex: Vertex):
    return vertex.pips

In [ ]:
def pips_eval(players: List[Player], board: Board):
    pips = [sum([settlement.pips for settlement in player.settlements]) for player in players]

    return pips

## Single Simulation

In [ ]:
sim = DraftSimulator(relative_custom_heuristic, simple_ruleout, num_pips)

sim.board.normal_randomize()
sim.board.populate_neighbors()
sim.board.populate_prospects()

In [ ]:
sim.reset()

# Using set board from the 2021 Catan US National Championship - final match
numbers = [4, 10, 5, 6, 2, 6, 3, 4, 11, 5, 9, 8, 9, 11, 10, 3, 12, 8]
resources = [Resource.WHEAT, Resource.SHEEP, Resource.SHEEP, Resource.ORE, Resource.BRICK, Resource.WOOD, Resource.WHEAT, Resource.SHEEP, Resource.WOOD, Resource.WOOD, Resource.BRICK, Resource.BRICK, Resource.WHEAT, Resource.ORE, Resource.DESERT, Resource.WHEAT, Resource.ORE, Resource.WOOD, Resource.SHEEP]

sim.board.set_board(resources, numbers)
sim.board.summarize()

start_time = time.time()
result = sim.run_top_k_mixed_max([0, 1, 2, 3, 3, 2, 1, 0], 4)
end_time = time.time()
print(f"Execution: {(end_time - start_time):.4f} seconds")

In [ ]:
# Print the JSON to view this match in the Catan viewer
print(json.dumps(sim.get_summary()))

## Full Simulation (19k drafts)

In [ ]:
sim = DraftSimulator(relative_custom_heuristic, simple_ruleout, num_pips)
sim.board.normal_randomize()
sim.board.populate_neighbors()
sim.board.populate_prospects()

## Run simulation
columns = ["ts1", "ts2", "ts3", "ts4", "tu1", "tu2", "tu3", "tu4", "tm1", "tm2", "tm3", "tm4", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", 
           "str1", "str2", "str3", "str4", "pr1", "pr2", "pr3", "pr4", "pr5", "pr6", "pr7", "pr8", "dl", "r1", "r2", "r3", "r4", "r5", "r6", 
           "r7", "r8", "wood1", "wood2", "wood3", "wood4", "brick1", "brick2", "brick3", "brick4", "sheep1", "sheep2", "sheep3", "sheep4",
           "wheat1", "wheat2", "wheat3", "wheat4", "ore1", "ore2", "ore3", "ore4", "pros1", "pros2", "pros3", "pros4", "pip1", "pip2", "pip3",
           "pip4", "pf1", "pf2", "pf3", "pf4"]
df = pd.DataFrame(columns=columns)

p1 = sim.players[0]
p2 = sim.players[1]
p3 = sim.players[2]
p4 = sim.players[3]

def sum_prospects(player: Player, board: Board):
    total = 0
    for settlement in player.settlements:
        for prospect in settlement.prospects:
            if prospect.available:
                total += prospect.pips
    return total

def get_strat(dist):
    metric_1 = min(dist[1:5]) * 5
    metric_2 = min([dist[1], dist[2]]) * 2
    metric_3 = min([dist[5] / 3, dist[3] / 2]) * 5
    top = max([metric_1, metric_2, metric_3])
    
    if top == metric_1:
        return 0
    if top == metric_2:
        return 1
    return 2

def pip_rank(index, chosen, summary):
    for index, option in enumerate(summary["history"][index]):
        if option["location"] == chosen:
            return k - index

def find_desert(board):
    for index, h in enumerate(board.hexes):
        if h.resource == 0:
            return index

def get_dist(player, board):
    pips = [0, 0, 0, 0, 0, 0]
    for settlement in player.settlements:
        for index, pip in enumerate(settlement.resource_pips):
            pips[index] += pip

    return pips

def get_pip_fit(dist):
    metric_1 = min(dist[1:5]) * 5
    metric_2 = min([dist[1], dist[2]]) * 2
    metric_3 = min([dist[5] / 3, dist[3] / 2]) * 5
    return max([metric_1, metric_2, metric_3])

def get_ts(pips, pip_fit, prospect_sum):
    total = []
    for i in range(len(pips)):
        total.append(pips[i] + 0.5 * pip_fit[i] + 0.25 * prospect_sum[i])
    return total

def get_tu(total):
    res = []
    for i in range(len(total)):
        m = np.mean(total[:i] + total[i + 1:])
        res.append(total[i] - m)
    return res

def get_tm(total):
    res = []
    for i in range(len(total)):
        m = np.max(total[:i] + total[i + 1:])
        res.append(total[i] - m)
    return res

rows = []
k = 4

beginning = time.time()

num_iters = 19000
for i in range(num_iters):
    start = time.time()
    sim.reset()
    sim.board.normal_randomize()
    result = sim.run_top_k_mixed_max([0, 1, 2, 3, 3, 2, 1, 0], k)
    summary = sim.get_summary()

    s1 = summary["chosen"][0]
    s2 = summary["chosen"][1]
    s3 = summary["chosen"][2]
    s4 = summary["chosen"][3]
    s5 = summary["chosen"][4]
    s6 = summary["chosen"][5]
    s7 = summary["chosen"][6]
    s8 = summary["chosen"][7]

    pr1 = pip_rank(0, s1, summary)
    pr2 = pip_rank(1, s2, summary)
    pr3 = pip_rank(2, s3, summary)
    pr4 = pip_rank(3, s4, summary)
    pr5 = pip_rank(4, s5, summary)
    pr6 = pip_rank(5, s6, summary)
    pr7 = pip_rank(6, s7, summary)
    pr8 = pip_rank(7, s8, summary)

    dl = find_desert(sim.board)

    r1 = summary["chosen_roads"][0]
    r2 = summary["chosen_roads"][1]
    r3 = summary["chosen_roads"][2]
    r4 = summary["chosen_roads"][3]
    r5 = summary["chosen_roads"][4]
    r6 = summary["chosen_roads"][5]
    r7 = summary["chosen_roads"][6]
    r8 = summary["chosen_roads"][7]

    dist1 = get_dist(p1, sim.board)
    dist2 = get_dist(p2, sim.board)
    dist3 = get_dist(p3, sim.board)
    dist4 = get_dist(p4, sim.board)

    pros1 = sum_prospects(p1, sim.board)
    pros2 = sum_prospects(p2, sim.board)
    pros3 = sum_prospects(p3, sim.board)
    pros4 = sum_prospects(p4, sim.board)

    pips = [sum([settlement.pips for settlement in player.settlements]) for player in sim.players]
    pip1 = pips[0]
    pip2 = pips[1]
    pip3 = pips[2]
    pip4 = pips[3]

    pf1 = get_pip_fit(dist1)
    pf2 = get_pip_fit(dist2)
    pf3 = get_pip_fit(dist3)
    pf4 = get_pip_fit(dist4)
    
    str1 = get_strat(dist1)
    str2 = get_strat(dist2)
    str3 = get_strat(dist3)
    str4 = get_strat(dist4)

    scores = get_ts([pip1, pip2, pip3, pip4], [pf1, pf2, pf3, pf4], [pros1, pros2, pros3, pros4])
    ts1 = scores[0]
    ts2 = scores[1]
    ts3 = scores[2]
    ts4 = scores[3]

    means = get_tu(scores)
    tu1 = means[0]
    tu2 = means[1]
    tu3 = means[2]
    tu4 = means[3]

    maxes = get_tm(scores)
    tm1 = maxes[0]
    tm2 = maxes[1]
    tm3 = maxes[2]
    tm4 = maxes[3]

    rows.append({
        "ts1": ts1,
        "ts2": ts2,
        "ts3": ts3,
        "ts4": ts4,
        "tu1": tu1,
        "tu2": tu2,
        "tu3": tu3,
        "tu4": tu4,
        "tm1": tm1,
        "tm2": tm2,
        "tm3": tm3,
        "tm4": tm4,
        "s1": s1,
        "s2": s2,
        "s3": s3,
        "s4": s4,
        "s5": s5,
        "s6": s6,
        "s7": s7,
        "s8": s8, 
        "str1": str1,
        "str2": str2,
        "str3": str3,
        "str4": str4,
        "pr1": pr1,
        "pr2": pr2,
        "pr3": pr3,
        "pr4": pr4,
        "pr5": pr5,
        "pr6": pr6,
        "pr7": pr7,
        "pr8": pr8,
        "dl": dl,
        "r1": r1,
        "r2": r2,
        "r3": r3,
        "r4": r4,
        "r5": r5,
        "r6": r6,
        "r7": r7,
        "r8": r8,
        "wood1": dist1[1],
        "wood2": dist2[1],
        "wood3": dist3[1],
        "wood4": dist4[1],
        "brick1": dist1[2],
        "brick2": dist2[2],
        "brick3": dist3[2],
        "brick4": dist4[2],
        "sheep1": dist1[4],
        "sheep2": dist2[4],
        "sheep3": dist3[4],
        "sheep4": dist4[4],
        "wheat1": dist1[3],
        "wheat2": dist2[3],
        "wheat3": dist3[3],
        "wheat4": dist4[3],
        "ore1": dist1[5],
        "ore2": dist2[5],
        "ore3": dist3[5],
        "ore4": dist4[5],
        "pros1": pros1,
        "pros2": pros2,
        "pros3": pros3,
        "pros4": pros4,
        "pip1": pip1,
        "pip2": pip2,
        "pip3": pip3,
        "pip4": pip4,
        "pf1": pf1,
        "pf2": pf2,
        "pf3": pf3,
        "pf4": pf4
    })
    end = time.time()
    if (i % 60 == 0):
        clear_output(wait=True)
    print(f"Completed iteration {i + 1} in {round(end - start, 2)} seconds. Total elapsed: {round(end - beginning, 2)} seconds")

df = pd.DataFrame(rows)
df.to_csv("results-10k.csv")

## Edge Indices (Based on the Vertices they Connect)

In [ ]:
board = Board()

for edge in board.edges:
    print(f"Index: {edge.index}\n{edge.vertices[1].location} - {edge.vertices[0].location}")